In [7]:

from roboflow import Roboflow
rf = Roboflow(api_key="QnnzsdXPEsvcmyZQsec6")
project = rf.workspace("ws-qwbuh").project("constellation-dsphi")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Constellation-1 in yolov11:: 100%|██████████| 4700/4700 [00:00<00:00, 10575.11it/s]


In [12]:
# !pip install ultralytics
# !pip install seaborn
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

In [13]:
dataset_path = "Constellation-1"

train_path = dataset_path + "/train"
valid_path = dataset_path + "/valid"
test_path = dataset_path + "/test"

train_imgs = train_path + "/images"
valid_imgs = valid_path + "/images"
test_imgs = test_path + "/images"

train_labels = train_path + "/labels"
valid_labels = valid_path + "/labels"
test_labels = test_path + "/labels"

In [14]:
# Get a random image from the train folder
image_files = [f for f in os.listdir(train_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
if not image_files:
    raise ValueError("No image files found in the training directory.")
random_image_file = random.choice(image_files)
image_path = os.path.join(train_path, random_image_file)

# Load image in grayscale
original = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply sigmoid contrast enhancement
# You can tweak gain and cutoff to control the steepness and center of the curve
def sigmoid_contrast(img, gain=10, cutoff=128):
    img = img.astype(np.float32)
    sigmoid = 255 / (1 + np.exp(-gain * (img - cutoff) / 255.0))
    return np.uint8(sigmoid)

transformed = sigmoid_contrast(original)

# Show original and transformed image
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(original, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Sigmoid Thresholded")
plt.imshow(transformed, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()

ValueError: No image files found in the training directory.